# ETL Credits:

## Libreries

In [17]:
import pandas as pd
import ast
import missingno as msno
import json


## Opening CSV Credits data:

In [18]:
df_credits_csv = pd.read_csv('D:\Data Scientist\CURSADO HENRY DTS09\PI\PI1_PT02\PI_ML_OPS_PT02\Datasets\Credits_datasets\Csvs Credits\credits_1.csv')

In [19]:
df_credits_csv.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


### Vamos a tener que explorarlo mas al apestoso csv credits 🤧

In [20]:
df_credits_csv.shape

(45476, 3)

In [21]:
df_credits_csv = df_credits_csv.drop_duplicates()

In [22]:
df_credits_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45439 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45439 non-null  object
 1   crew    45439 non-null  object
 2   id      45439 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


elimino la ultima fila que esta vacia

In [23]:
df_credits_csv = df_credits_csv.drop(df_credits_csv.index[-1])

### Separo la columna id y la guardo aparte para asegurarla, debido a que parece estar ok

In [24]:
df_movie_credits_id = pd.DataFrame()
df_movie_credits_id['id']= df_credits_csv['id']
df_movie_credits_id = df_movie_credits_id.drop_duplicates()
print(type(df_movie_credits_id))

<class 'pandas.core.frame.DataFrame'>


### Continuo Explorando las dos primeras columnas, cast y crew

#### Cast:

In [25]:
df_credits_cast = df_credits_csv[['cast','id']]

print(df_credits_cast)

                                                    cast      id
0      [{'cast_id': 14, 'character': 'Woody (voice)',...     862
1      [{'cast_id': 1, 'character': 'Alan Parrish', '...    8844
2      [{'cast_id': 2, 'character': 'Max Goldman', 'c...   15602
3      [{'cast_id': 1, 'character': "Savannah 'Vannah...   31357
4      [{'cast_id': 1, 'character': 'George Banks', '...   11862
...                                                  ...     ...
45470  [{'cast_id': 1, 'character': 'Sir Robert Hode'...   30840
45471  [{'cast_id': 0, 'character': '', 'credit_id': ...  439050
45472  [{'cast_id': 1002, 'character': 'Sister Angela...  111109
45473  [{'cast_id': 6, 'character': 'Emily Shaw', 'cr...   67758
45474  [{'cast_id': 2, 'character': '', 'credit_id': ...  227506

[45438 rows x 2 columns]


In [26]:
df_credits_cast = df_credits_cast.drop_duplicates()

In [27]:
print(df_credits_cast.iloc[1:2])

                                                cast    id
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...  8844


In [36]:
import json

iteracion = 0
df_cast_cleaned = pd.DataFrame()

# Iterar a través de la columna 'cast' en df_credits_cast
for row in df_credits_cast['cast']:
    try:
        # Utilizar json.loads para convertir la representación de cadena en una lista de diccionarios
        list_of_dicts = json.loads(row.replace("'", "\""))

        # Verificar si el 'id' de la película existe en la columna 'id' de df_movie_credits_id
        if df_movie_credits_id['id'].isin([d['id'] for d in list_of_dicts]).any():
            # Crear el DataFrame con los datos del cast
            cast_df = pd.DataFrame(list_of_dicts)

            # Obtener el 'id' de la película de df_movie_credits_id
            movie_id = df_movie_credits_id['id'].iloc[iteracion]

            # Agregar el 'movie_id' al DataFrame cast_df
            cast_df['movie_id'] = movie_id

            # Unificar los DataFrames utilizando concat
            df_cast_cleaned = pd.concat([df_cast_cleaned, cast_df], ignore_index=True)

            # Incrementar el contador de iteración
            iteracion += 1
        else:
            # Si el 'id' de la película no existe en df_movie_credits_id, omitimos esta película
            pass
    except json.JSONDecodeError:
        # Omitir las filas que no se pueden decodificar como JSON
        pass


In [39]:
cast_df

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
0,1,Sir Robert Hode,52fe44439251416c9100a887,2,29459,Patrick Bergin,0,/mxFum2Fr5YWD6kDQYb5u7EEvuva.jpg,111815
1,2,Maid Marian,52fe44439251416c9100a88b,1,139,Uma Thurman,1,/6SuOc2R7kXjq3Em24KTNDW9qblJ.jpg,111815
2,3,Little John,52fe44439251416c9100a88f,2,18616,David Morrissey,2,/y8L9zlrIzsAwu2Hufol6KqoDS4I.jpg,111815
3,4,Sir Miles Folcanet,52fe44439251416c9100a893,2,920,Jürgen Prochnow,3,/aar3n9NEQdwWVvPOne3kLLXm75N.jpg,111815
4,14,Baron Roger Daguerre,58207d0d92514168ad013b03,0,1924,Jeroen Krabbé,4,/A5Mks6zi6Xzt9A8fxBWc3wdlvhx.jpg,111815


In [40]:
df_cast_cleaned.shape

(53558, 9)

In [41]:
df_cast_cleaned

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
0,2,Max Goldman,52fe466a9251416c75077a8d,2,6837,Walter Matthau,0,/xJVkvprOnzP5Zdh5y63y8HHniDZ.jpg,862
1,3,John Gustafson,52fe466a9251416c75077a91,2,3151,Jack Lemmon,1,/chZmNRYMtqkiDlatprGDH4BzGqG.jpg,862
2,4,Ariel Gustafson,52fe466a9251416c75077a95,1,13567,Ann-Margret,2,/jx5lTaJ5VXZHYB52gaOTAZ9STZk.jpg,862
3,5,Maria Sophia Coletta Ragetti,52fe466a9251416c75077a99,1,16757,Sophia Loren,3,/emKLhbji1c7BjcA2DdbWf0EP9zH.jpg,862
4,6,Melanie Gustafson,52fe466a9251416c75077a9d,1,589,Daryl Hannah,4,/4LLmp6AQdlj6ueGCRbVRSGvvFSt.jpg,862
...,...,...,...,...,...,...,...,...,...
53553,1,Sir Robert Hode,52fe44439251416c9100a887,2,29459,Patrick Bergin,0,/mxFum2Fr5YWD6kDQYb5u7EEvuva.jpg,111815
53554,2,Maid Marian,52fe44439251416c9100a88b,1,139,Uma Thurman,1,/6SuOc2R7kXjq3Em24KTNDW9qblJ.jpg,111815
53555,3,Little John,52fe44439251416c9100a88f,2,18616,David Morrissey,2,/y8L9zlrIzsAwu2Hufol6KqoDS4I.jpg,111815
53556,4,Sir Miles Folcanet,52fe44439251416c9100a893,2,920,Jürgen Prochnow,3,/aar3n9NEQdwWVvPOne3kLLXm75N.jpg,111815


In [ ]:
df_credits_cast.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45431 entries, 0 to 45474
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45431 non-null  object
 1   id      45431 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


#### Crew:

______________________________________________________________________